In [48]:
import pandas as pd
import csv
import numpy as np
from datetime import date, timedelta, datetime
import pymysql
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

In [49]:
# SQL functions
import pymysql
endpoint = 'ta10projectinstance1.c0m59tujlbqm.ap-southeast-2.rds.amazonaws.com'
username = 'admin'
password = 'fit5120ta10'
database_name = 'fit5120_i3_schema'
client_flag = 'CLIENT.MULTI_STATEMENTS'

connection = pymysql.connect(host = endpoint, user = username, passwd = password, db = database_name)

# Function used to clear all entries in a single table considering no indexes or Foreign keys are present
def truncate_handler(table):
    cursor = connection.cursor()
    cursor.execute("""TRUNCATE {}""".format(table))
    connection.commit()
    return 'table {} has been truncated'.format(table)

def country_region_handler(values):
    query = """INSERT INTO country_region (country_region_id, country_code, region_name) VALUES (%s, %s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def country_handler(values):
    query = """INSERT INTO country (country_name, country_code) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def symptom_handler(values):
    query = """INSERT INTO symptom (symp_id, symp_name) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_details_handler(values):
    query = """INSERT INTO case_details (case_id, symp_id) VALUES (%s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def case_handler(values):
    query = """INSERT INTO `case` (case_id, case_status, case_date, case_gender, country_region_id, case_death_date) VALUES (%s, %s, %s, %s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'import successful {}'.format(values)

def prediction_handler(values):
    query = """INSERT INTO predictions (country_region_id, pred_date, pred_count) VALUES (%s, %s, %s)"""

    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    return 'predictions table has been updated'

def query(query):
    sql_query = """{}""".format(query)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    return cursor.fetchall()

In [50]:
def prediction():
    vic_aug = pd.read_csv("https://raw.githubusercontent.com/owid/monkeypox/main/owid-monkeypox-data.csv") # reading the file
    vic_aug=vic_aug.query('location=="Australia"') #filtering for australia
    vic_aug=vic_aug.reset_index() #reseting index
    vic_aug = vic_aug[['date', 'new_cases_smoothed']] #subselection of attributes
    EndDate = date.today() + timedelta(days=5)#getting the days from current datetime
    sdate=vic_aug.iloc[0,:]['date']#getting the start date
    sdate=datetime.strptime(sdate,'%Y-%m-%d').date()#converting to datetime format
    time_range=pd.date_range(sdate,EndDate-timedelta(days=1),freq='d')#retrieving the dates 
    time_range=time_range.to_frame(index=False, name='date') #converting it to a dataframe
    time_range['date']=time_range['date'].astype(str) #Converting to string datatype
    merged_df=pd.merge(time_range,vic_aug,on='date',how='left') #left joining
    merged_df["new_cases_smoothed"].fillna(0, inplace=True)#replacing predicted values to initially 0
    step_fit=auto_arima(merged_df['new_cases_smoothed'],trace=True,suppress_warnings=True) #stepwise function for auto arima
    model = ARIMA(merged_df.new_cases_smoothed, order=step_fit.get_params().get("order")) # fetching the best parameters
    #model=ARIMA(train[''],order=(3,1,0))
    model=model.fit()
    train=merged_df.iloc[:-100] #splitting training and testing
    test=merged_df.iloc[-100:]
    start=len(test)
    end=len(train)+len(test)-1
    pred=model.predict(start=start,end=end,typ='levels') #predicting the values
    series1 = pred.to_frame()
    series1['predicted_mean'] = series1['predicted_mean'].abs() #taking absolute values
    series1['predicted_mean']=series1['predicted_mean'].apply(np.floor)  #flooring the values
    series1=series1.iloc[-5:]
    test=test.iloc[-5:]
    test['pred']=series1['predicted_mean']
    test=test[['date','pred']]
    return test #returning the dataframe

Main function to run the model, truncate the prediction table and push the new data

In [51]:
def main():
    # runs the model
    df = prediction() 
    # renames all the columns to the appropriate format for the relational database
    df = df.rename(columns = {'date': 'pred_date', 'pred':'pred_count'})
    # Adds the country_region_id by default it will be AUS_0 due to naming convention
    df['country_region_id'] = 'AUS_0'
    # truncates the prediction table
    truncate_handler('predictions')
    # uploads the new data into the predictions table
    p_df = df.copy()
    for row in range(0,p_df.shape[0]):
        in_tuple = tuple(p_df.iloc[row, :])
        prediction_handler((in_tuple[2], in_tuple[0], in_tuple[1]))
    return 'prediction table has been updated'
main()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=161.064, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=181.187, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=176.857, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=173.166, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=179.190, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=173.880, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=173.767, Time=0.03 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=177.644, Time=0.07 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=162.585, Time=0.13 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=172.633, Time=0.03 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=175.853, Time=0.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=175.644, Time=0.02 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=165.003, Time=0.13 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=159.067, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=171.8

Query to check the new data in the prediction table

In [47]:
query('SELECT * FROM predictions')

(('AUS_0', '2022-10-08', 0),
 ('AUS_0', '2022-10-09', 0),
 ('AUS_0', '2022-10-10', 0),
 ('AUS_0', '2022-10-11', 0),
 ('AUS_0', '2022-10-12', 0))